<a href="https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_3miceDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox Demo on 3 mice data
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628250004229-KVYD7JJVHYEFDJ32L9VJ/DLClogo2021.jpg?format=1000w)

https://github.com/DeepLabCut/DeepLabCut

### This notebook illustrates how to use COLAB for a multi-animal DeepLabCut (maDLC) Demo 3 mouse project:
- load our mini-demo data that includes a pretrained model and unlabeled video.
- analyze a novel video.
- assemble animals and tracklets.
- create quality check plots and video.

### To create a full maDLC pipeline please see our full docs: https://deeplabcut.github.io/DeepLabCut/docs/intro.html 
- Of interest is a full how-to for maDLC: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html
- a quick guide to maDLC: https://deeplabcut.github.io/DeepLabCut/docs/tutorial.html
- a demo COLAB for how to use maDLC on your own data: https://github.com/DeepLabCut/DeepLabCut/blob/master/examples/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb

### To get started, please go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
# Install the latest DeepLabCut version:
!pip install https://github.com/DeepLabCut/DeepLabCut/archive/master.zip

No information needs edited in the cells below, you can simply click run on each:

### Download our Demo Project from our server:

In [ ]:
# Download our demo project:
import urllib.request
from io import BytesIO
from zipfile import ZipFile

def unzip_from_url(url, dest_folder=''):
    # Directly extract files without writing the archive to disk
    resp = urllib.request.urlopen(url)
    with ZipFile(BytesIO(resp.read())) as zf:
        zf.extractall(path=dest_folder)


project_url = "http://deeplabcut.rowland.harvard.edu/datasets/demo-me-2021-07-14.zip"
unzip_from_url(project_url, "/content")

## Analyze a novel 3 mouse video with our maDLC DLCRNet, pretrained on 3 mice data (i.e., here you extract detections and association costs):

In [ ]:
import deeplabcut as dlc
import os

project_path = "/content/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")
video = os.path.join(project_path, "videos", "videocompressed1.mp4")

dlc.analyze_videos(config_path,[video], shuffle=0, videotype="mp4",auto_track=False )

### Next, you compute the local, spatio-temporal grouping and track body part assemblies frame-by-frame:

In [ ]:
TRACK_METHOD = "ellipse"  # Could also be "box", but "ellipse" was found to be more robust on this dataset.

dlc.convert_detections2tracklets(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    track_method=TRACK_METHOD,
    ignore_bodyparts=["tail1", "tail2", "tailend"],  # Some body parts can optionally be ignored during tracking for better assembly (but they are used later)
)

### Reconstruct full animal trajectories (tracks from tracklets):

In [ ]:
dlc.stitch_tracklets(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    track_method=TRACK_METHOD,
    n_tracks=3,
)

## Create a pretty video output:

In [ ]:
#Filter the predictions to remove small jitter, if desired:
dlc.filterpredictions(config_path, 
                                 [video], 
                                 shuffle=0,
                                 videotype='mp4', 
                                 track_method = TRACK_METHOD)

dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
    track_method=TRACK_METHOD,
)

Now, on the left panel if you click the folder icon, you will see the project folder "demo-me.."; click on this and go into "videos" and you can find the "..._id_labeled.mp4" video, which you can double-click on to download and inspect!

### Create Plots of your data:

> after running, you can look in "videos", "plot-poses" to check out the trajectories! (sometimes you need to click the folder refresh icon to see it). Within the folder, for example, see plotmus1.png to vide the bodyparts over time vs. pixel position.



In [ ]:
dlc.plot_trajectories(config_path, [video], shuffle=0,videotype='mp4', track_method=TRACK_METHOD)